# Data import

This notebook organizes the import of data to the DWC dabase. First **Sites** of type **flow** are imported.

In [1]:
from ews.models import *
import pandas as pd
import geojson
import numpy as np


In [121]:
#Site.objects.all().delete()

(36, {'ews.Site': 36})

In [2]:
d = pd.read_csv("D:/DWC_plattform/metadata_flownames_corrected.csv")

In [3]:
x = d.duplicated(subset=['SiteName']) == False

In [4]:
d = d[x]

In [5]:
d.duplicated(subset=['SiteName']).sum()

0

,Category,SiteID,SiteName,SiteReferencementName,SiteCity,SiteType,SourceID,SiteLatitude(y),SiteLongitude(x)
7,Flow,96,ru_de_nelse,GG,NaN,Network,SSD/SIAAP,48.86449,2.58539


In [28]:
for index, row in d.iterrows():
    loc = Site()
    loc.name = row['SiteName']
    loc.ref_name = row['SiteReferencementName']
    loc.geom = {"coordinates": [row['SiteLongitude(x)'],row['SiteLatitude(y)']], "type": "Point"}
    loc.feature_type = FeatureType.objects.get(name = row['SiteType'])
    loc.owner = User.objects.get(id = 1)
    try:
        loc.save()
    except:
        print(loc)

In [29]:
flows = pd.read_csv('D:/DWC_plattform/daily_flows.csv',parse_dates = True, index_col = "date")

In [30]:
flows.head()

,day,month,year,site,M
date,,,,,
2015-05-01,1,5,2015,canal_chesney,0.099604
2015-05-01,1,5,2015,centre_urbain,0.265285
2015-05-01,1,5,2015,maltournée,0.105615
2015-05-01,1,5,2015,neuilly_gagny,0.038722
2015-05-01,1,5,2015,ru_de_nelse,0.001000


In [132]:
#sites = flows.site.unique()

In [133]:
#select = flows[flows["site"]==sites[0]]

In [134]:
#daily = select.resample("6H").mean()

In [135]:
#aggregates =flows.groupby("site").resample("6H").mean()

In [136]:
#d = aggregates.reset_index()

In [31]:
flows.M = flows.M.round(3)

In [32]:
flows.sort_values(by = "M")

,day,month,year,site,M
date,,,,,
2017-09-28,28,9,2017,maltournée,-9999.0
2017-08-26,26,8,2017,maltournée,-9999.0
2017-09-08,8,9,2017,maltournée,-9999.0
2017-09-04,4,9,2017,maltournée,-9999.0
2017-08-23,23,8,2017,maltournée,-9999.0
...,...,...,...,...,...
2018-10-31,31,10,2018,lande_sud_plage,NaN
2018-10-31,31,10,2018,marc_sangnier,NaN
2019-10-31,31,10,2019,8_mai_45,NaN


In [33]:
flows

,day,month,year,site,M
date,,,,,
2015-05-01,1,5,2015,canal_chesney,0.100
2015-05-01,1,5,2015,centre_urbain,0.265
2015-05-01,1,5,2015,maltournée,0.106
2015-05-01,1,5,2015,neuilly_gagny,0.039
2015-05-01,1,5,2015,ru_de_nelse,0.001
...,...,...,...,...,...
2019-10-31,31,10,2019,ru_du_marais,0.000
2019-10-31,31,10,2019,saint_baudile,0.000
2019-10-31,31,10,2019,seine_valenton_wwtp,10904.062


In [34]:
d = flows

In [35]:
d.rename(columns = {"M": "value"}, inplace = True) 

In [36]:
d

,day,month,year,site,value
date,,,,,
2015-05-01,1,5,2015,canal_chesney,0.100
2015-05-01,1,5,2015,centre_urbain,0.265
2015-05-01,1,5,2015,maltournée,0.106
2015-05-01,1,5,2015,neuilly_gagny,0.039
2015-05-01,1,5,2015,ru_de_nelse,0.001
...,...,...,...,...,...
2019-10-31,31,10,2019,ru_du_marais,0.000
2019-10-31,31,10,2019,saint_baudile,0.000
2019-10-31,31,10,2019,seine_valenton_wwtp,10904.062


In [37]:
d.loc[d["value"] < 0, ["value"]] = np.nan

In [38]:
d = d.dropna()

In [39]:
#FeatureData.objects.all().delete()

In [40]:
y=d.sort_values(by = "value", ascending = False).tail(3)

In [41]:
d = d.reset_index()

In [43]:
d = d[d.site == "ru_de_nelse"]

In [44]:
d

,date,day,month,year,site,value
4,2015-05-01,1,5,2015,ru_de_nelse,0.001
20,2016-05-01,1,5,2016,ru_de_nelse,0.001
52,2017-05-01,1,5,2017,ru_de_nelse,0.001
113,2018-05-01,1,5,2018,ru_de_nelse,0.001
177,2019-05-01,1,5,2019,ru_de_nelse,0.001
...,...,...,...,...,...,...
49133,2015-10-31,31,10,2015,ru_de_nelse,0.001
49149,2016-10-31,31,10,2016,ru_de_nelse,0.001
49182,2017-10-31,31,10,2017,ru_de_nelse,0.001
49241,2018-10-31,31,10,2018,ru_de_nelse,0.001


In [45]:
for index, row in d.iterrows():
    dpoint = FeatureData()
    dpoint.date = row["date"]
    dpoint.value = row["value"]
    dpoint.site = Site.objects.get(name = row['site'])
    dpoint.save()
    
    


C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-05-01 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-09-02 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-09-02 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-09-02 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-09-03 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-09-03 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2019-09-03 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-09-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-09-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-09-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-09-06 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-09-06 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-09-06 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

...

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-06-24 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-06-24 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-06-24 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-05-26 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2019-05-26 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-06-26 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-10-27 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2019-10-27 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-05-28 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2018-10-28 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2019-10-28 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-05-29 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe

C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2015-09-30 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2016-09-30 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField FeatureData.date received a naive datetime (2017-09-30 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Users\wolfg\miniconda3\envs\DWC\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Fe